In [82]:
# https://junpyopark.github.io/road-network-construction-1/

In [1]:
import shapefile #the pyshp module : Should install pyshp module.
import pandas as pd
from pyproj import Proj, transform # Should install pyproj module.
import networkx as nx

In [2]:
#  encoding = 'cp949'

In [3]:
# read data (Copy all files from nodelink into nodelink folder: I made it.)
# using old_data
shp_path_node = './nodelink_190709/MOCT_NODE.shp'
sf_node = shapefile.Reader(shp_path_node, encoding = 'cp949')
shp_path_link = './nodelink_190709/MOCT_LINK.shp'
sf_link = shapefile.Reader(shp_path_link, encoding = 'cp949')

In [4]:
# construct pandas dataframe

#grab the shapefile's field names
# node
fields_node = [x[0] for x in sf_node.fields][1:]
records_node = sf_node.records()
shps = [s.points for s in sf_node.shapes()] # node has coordinate data.
# link
fields_link = [x[0] for x in sf_link.fields][1:]
records_link = sf_link.records()

#write the records into a dataframe
# node
node_dataframe = pd.DataFrame(columns=fields_node, data=records_node)
#add the coordinate data to a column called "coords"
node_dataframe = node_dataframe.assign(coords=shps)
# link
link_dataframe = pd.DataFrame(columns=fields_link, data=records_link)

In [5]:
node_dataframe[1:5]# Show first 5 items.

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,coords
1,3330016500,104,벌교터널,0,,korea,5,,333,OEXT3330016500,,"[[229535.21800000034, 250194.52040000074]]"
2,3330015600,104,벌교터널,0,,korea,5,,333,OEXT3330015600,,"[[229465.27020000014, 249416.52300000004]]"
3,3330014900,101,벌교교차로,0,,korea,5,,333,OEXT3330014900,,"[[229455.64099999983, 249249.00129999965]]"
4,3330014600,101,벌교교차로,0,,korea,5,,333,OEXT3330014600,,"[[229412.60429999977, 249169.92640000023]]"


In [6]:
node_dataframe[node_dataframe['STNL_REG'] == str(100)][1:5]

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,coords
98087,1000017800,101,파크뷰타워,1,,,5,11110,100,,,"[[197248.12399999984, 552890.1248000003]]"
98088,1000018000,101,적선현대빌딩,1,,,5,11110,100,,,"[[197587.4173999997, 552862.0014999993]]"
98089,1000007300,101,세종로사거리_태평로_진,0,,,5,11110,100,,,"[[197963.9944000002, 552227.1275999993]]"
98090,1000007600,101,세종로사거리_일반국도6,0,,,5,11110,100,,,"[[197897.61730000004, 552286.2988000009]]"


In [7]:
node_dataframe.dtypes # Check data types of columns.

NODE_ID      object
NODE_TYPE    object
NODE_NAME    object
TURN_P       object
REMARK       object
USER_ID      object
WORKSTATE    object
DEPT_CODE    object
STNL_REG     object
TMPID        object
UPLOAD_ID    object
coords       object
dtype: object

In [8]:
# Data restriction
range_STNL_REG=range(100, 124) # STNL_REG for Incheon
df_node = pd.DataFrame()
df_link = pd.DataFrame()
for ii in range_STNL_REG:
    res_node = node_dataframe[node_dataframe['STNL_REG'] == str(ii) ] # STNL_REG is not int.
    res_link = link_dataframe[link_dataframe['STNL_REG'] == str(ii) ]
    df_node = pd.concat([df_node,res_node],ignore_index=True) # marge data
    df_link = pd.concat([df_link,res_link],ignore_index=True)

In [9]:
# Change node name in korean 
for idx,row in df_node.iterrows():
    if type(row['NODE_NAME']) == bytes :
        # row['NODE_NAME'] = row['NODE_NAME'].decode('euc_kr')
        row['NODE_NAME'] = row['NODE_NAME'].decode('cp949')

In [10]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186')
outProj= Proj(init = 'epsg:4326')
latitude = []
longitude= []
for idx,row in df_node.iterrows():
    x,y  = row.coords[0][0],row.coords[0][1]  # korea 2000 좌표계
    nx,ny = transform(inProj,outProj,x,y)     # 새로운 좌표계    
    latitude.append(ny)
    longitude.append(nx)
df_node['latitude'] = latitude
df_node['longitude']= longitude
del df_node['coords'] # delete coords

In [11]:
# Change column name to draw network in Gephi
df_node.rename(columns={'NODE_ID':'Id'},inplace = True)
df_link.rename(columns={'F_NODE':'Source','T_NODE':'Target'},inplace = True)

In [12]:
print(len(df_node))

7462


In [13]:
df_node.head()

,Id,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,latitude,longitude
0,1000029100,101,비봉길-진흥로진입로,1,,,5,11110,100,,,37.608424,126.956750
1,1000017800,101,파크뷰타워,1,,,5,11110,100,,,37.575554,126.968848
2,1000018000,101,적선현대빌딩,1,,,5,11110,100,,,37.575301,126.972689
3,1000007300,101,세종로사거리_태평로_진,0,,,5,11110,100,,,37.569582,126.976953
4,1000007600,101,세종로사거리_일반국도6,0,,,5,11110,100,,,37.570115,126.976202


In [14]:
df_link[1:5]

,LINK_ID,Source,Target,ROAD_USE,LANES,ROAD_RANK,ROAD_NAME,MULTI_LINK,CONNECT,MAX_SPD,...,WORKSTATE,DEPT_CODE,STNL_REG,ROAD_TYPE,ROAD_NO,TMPID,UPLOAD_ID,SOSFNODEID,SOSTNODEID,SHAPE_STLe
1,1000044700,1010034900,1000031400,1,1.0,104,종로40길,0,000,60.0,...,5,11110,100,003,-,TMP201410L10109,,,TMP201410N10133,33.051833
2,1000044800,1000031400,1010034900,0,1.0,104,종로40길,0,000,60.0,...,5,11110,100,003,-,TMP201410L10110,,TMP201410N10133,,33.051737
3,1000044900,1000031200,1000031300,0,1.0,103,종로,0,000,30.0,...,5,11110,100,000,6,TMP201410L10111,,TMP201410N10180,TMP201410N10179,82.135332
4,1000005602,1000031200,1000011000,1,5.0,107,창경궁로,0,000,60.0,...,5,11110,100,000,-,,,TMP201410N10180,,57.586025


In [15]:
df_node.to_csv('Seoul_nodes_190709.csv') # node list
df_link.to_csv('Seoul_links_190709.csv') # edge(=link) list

In [16]:
df_link.columns

Index(['LINK_ID', 'Source', 'Target', 'ROAD_USE', 'LANES', 'ROAD_RANK',
       'ROAD_NAME', 'MULTI_LINK', 'CONNECT', 'MAX_SPD', 'REST_VEH', 'REST_W',
       'REST_H', 'REMARK', 'USER_ID', 'WORKSTATE', 'DEPT_CODE', 'STNL_REG',
       'ROAD_TYPE', 'ROAD_NO', 'TMPID', 'UPLOAD_ID', 'SOSFNODEID',
       'SOSTNODEID', 'SHAPE_STLe'],
      dtype='object')

## folium

In [17]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

In [18]:
nodes = pd.read_csv('Seoul_nodes_190709.csv')
links = pd.read_csv('Seoul_links_190709.csv')

In [19]:
nodes = nodes[['Id','NODE_NAME','latitude','longitude']]

In [20]:
links = links[['Source','Target']]
links.head()

,Source,Target
0,1000010900,1000011600
1,1010034900,1000031400
2,1000031400,1010034900
3,1000031200,1000031300
4,1000031200,1000011000


In [21]:
source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in incheon_id
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in incheon_id
# source_in and target_in are boolean type pandas.Series which contains True or False
Seoul_links = links[source_in & target_in] # contain if both target and source are contained in incheon_id

In [24]:
G = nx.Graph()
# R is the Earth's radius
R = 6371e3

for idx,row in nodes.iterrows():
    # add node to Graph G
    G.add_node(row['Id'],Label=row['NODE_NAME'],latitude=row['latitude'], longitude=row['longitude'])

for idx,row in Seoul_links.iterrows():
    ## Calculate the distance between Source and Target Nodes
    lon1 = float(nodes[nodes['Id'] == row['Source']]['longitude'] * np.pi/180)
    lat1 = float(nodes[nodes['Id'] == row['Source']]['latitude'] * np.pi/180)
    lon2 = float(nodes[nodes['Id'] == row['Target']]['longitude'] * np.pi/180)
    lat2 = float(nodes[nodes['Id'] == row['Target']]['latitude'] * np.pi/180)
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    a = np.sin(d_lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon/2) ** 2
    c = 2 * np.arctan2(a**0.5, (1-a) ** 0.5)
    d = R * c
    
    # Link attribute : 'Source', 'Target' and weight = 'Length between them'
    G.add_edge(row['Source'],row['Target'],weight = d)

In [25]:
print(nx.info(G)) # 네트워크 정보 확인 

Name: 
Type: Graph
Number of nodes: 7462
Number of edges: 12027
Average degree:   3.2235


In [26]:
nx.write_gexf(G,'Seoul_2015.gexf')

In [27]:
# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
std_point

(37.60842435049385, 126.95675015496396)

In [28]:
map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for ix, row in nodes.iterrows():
    location = (row['latitude'], row['longitude']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=G.degree[row['Id']] * 30, # 지름이 degree에 비례하도록 설정
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)
map_osm.save('Seoul_road_1.html')

In [29]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in Seoul_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm.save('Seoul_road_2.html')